In [1]:
import argparse
import os
import pathlib
import sys
import uuid

import duckdb
import pandas as pd
from arg_parsing_utils import parse_args
from cytotable import convert, presets
from notebook_init_utils import bandicoot_check, init_notebook
from parsl.config import Config
from parsl.executors import HighThroughputExecutor

root_dir, in_notebook = init_notebook()

profile_base_dir = bandicoot_check(
    pathlib.Path(os.path.expanduser("~/mnt/bandicoot/NF1_organoid_data")).resolve(),
    root_dir,
)

In [2]:
if not in_notebook:
    args = parse_args()
    well_fov = args["well_fov"]
    patient = args["patient"]
    image_based_profiles_subparent_name = args["image_based_profiles_subparent_name"]

else:
    patient = "NF0014_T1"
    well_fov = "C4-2"
    image_based_profiles_subparent_name = "image_based_profiles"

In [3]:
input_sqlite_file = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/0.converted_profiles/{well_fov}/{well_fov}.duckdb"
).resolve(strict=True)
destination_sc_parquet_file = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/0.converted_profiles/{well_fov}/sc_profiles_{well_fov}.parquet"
).resolve()
destination_organoid_parquet_file = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/{image_based_profiles_subparent_name}/0.converted_profiles/{well_fov}/organoid_profiles_{well_fov}.parquet"
).resolve()
destination_sc_parquet_file.parent.mkdir(parents=True, exist_ok=True)
dest_datatype = "parquet"

In [4]:
# show the tables
with duckdb.connect(input_sqlite_file) as con:
    tables = con.execute("SHOW TABLES").fetchdf()
    print(tables)
    nuclei_table = con.sql("SELECT * FROM Nuclei").df()
    cells_table = con.sql("SELECT * FROM Cell").df()
    cytoplasm_table = con.sql("SELECT * FROM Cytoplasm").df()
    organoid_table = con.sql("SELECT * FROM Organoid").df()

        name
0       Cell
1  Cytoplasm
2     Nuclei
3   Organoid


In [5]:
nuclei_id_set = set(nuclei_table["object_id"].to_list())
cells_id_set = set(cells_table["object_id"].to_list())
cytoplasm_id_set = set(cytoplasm_table["object_id"].to_list())
# find the intersection of the three sets
intersection_set = nuclei_id_set.intersection(cells_id_set, cytoplasm_id_set)
# keep only the rows in the three tables that are in the intersection set
nuclei_table = nuclei_table[nuclei_table["object_id"].isin(intersection_set)]
cells_table = cells_table[cells_table["object_id"].isin(intersection_set)]
cytoplasm_table = cytoplasm_table[cytoplasm_table["object_id"].isin(intersection_set)]

In [6]:
# connect to DuckDB and register the tables
with duckdb.connect() as con:
    con.register("nuclei", nuclei_table)
    con.register("cells", cells_table)
    con.register("cytoplasm", cytoplasm_table)
    # Merge them with SQL
    merged_df = con.execute("""
        SELECT *
        FROM nuclei
        LEFT JOIN cells USING (object_id)
        LEFT JOIN cytoplasm USING (object_id)
    """).df()

In [7]:
# save the organoid data as parquet
print(f"Final organoid data shape: {merged_df.shape}")
organoid_table.to_parquet(destination_organoid_parquet_file, index=False)
organoid_table.head()

Final organoid data shape: (30, 7690)


,object_id,image_set,Area.Size.Shape_Organoid_VOLUME,Area.Size.Shape_Organoid_CENTER.X,Area.Size.Shape_Organoid_CENTER.Y,Area.Size.Shape_Organoid_CENTER.Z,Area.Size.Shape_Organoid_BBOX.VOLUME,Area.Size.Shape_Organoid_MIN.X,Area.Size.Shape_Organoid_MAX.X,Area.Size.Shape_Organoid_MIN.Y,...,Texture_Organoid_Mito_Difference.Entropy_256.3,Texture_Organoid_Mito_Difference.Variance_256.3,Texture_Organoid_Mito_Entropy_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_Mito_Inverse.Difference.Moment_256.3,Texture_Organoid_Mito_Sum.Average_256.3,Texture_Organoid_Mito_Sum.Entropy_256.3,Texture_Organoid_Mito_Sum.Variance_256.3,Texture_Organoid_Mito_Variance_256.3
0,32,C4-2,21335462.0,665.872681,558.247925,14.812983,33761508.0,167.0,1171.0,38.0,...,1.435332,0.0023,2.867987,-0.477698,0.913241,0.818414,8.453075,2.186957,198.758346,50.705657


In [8]:
# drop columns that end with _x or _y lowercase
merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith(("_x", "_y"))]

In [9]:
print(f"Final merged single cell dataframe shape: {merged_df.shape}")
# save the sc data as parquet
merged_df.to_parquet(destination_sc_parquet_file, index=False)
merged_df.head()

Final merged single cell dataframe shape: (30, 7690)


,object_id,image_set,Area.Size.Shape_Nuclei_VOLUME,Area.Size.Shape_Nuclei_CENTER.X,Area.Size.Shape_Nuclei_CENTER.Y,Area.Size.Shape_Nuclei_CENTER.Z,Area.Size.Shape_Nuclei_BBOX.VOLUME,Area.Size.Shape_Nuclei_MIN.X,Area.Size.Shape_Nuclei_MAX.X,Area.Size.Shape_Nuclei_MIN.Y,...,Texture_Cytoplasm_Mito_Difference.Entropy_256.3,Texture_Cytoplasm_Mito_Difference.Variance_256.3,Texture_Cytoplasm_Mito_Entropy_256.3,Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.1_256.3,Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.2_256.3,Texture_Cytoplasm_Mito_Inverse.Difference.Moment_256.3,Texture_Cytoplasm_Mito_Sum.Average_256.3,Texture_Cytoplasm_Mito_Sum.Entropy_256.3,Texture_Cytoplasm_Mito_Sum.Variance_256.3,Texture_Cytoplasm_Mito_Variance_256.3
0,2,C4-2,5995.0,847.588806,74.060219,0.000000,7304.0,804.0,892.0,32.0,...,0.013157,0.003885,0.016155,-0.453011,0.096733,0.999189,0.148783,0.014542,29.614068,8.670842
1,18,C4-2,28163.0,508.470001,255.540817,1.012534,38610.0,453.0,563.0,196.0,...,0.029950,0.003875,0.038031,-0.417001,0.140496,0.998156,0.070246,0.033772,2.698782,0.812618
2,28,C4-2,125455.0,912.757629,419.849945,16.152229,192600.0,858.0,965.0,358.0,...,0.098986,0.003830,0.131344,-0.502604,0.289441,0.992755,0.430457,0.116722,23.309541,6.457143
3,29,C4-2,66761.0,409.968811,652.169250,4.913228,973760.0,355.0,534.0,201.0,...,0.026490,0.003877,0.032596,-0.397218,0.125592,0.998214,0.241243,0.029414,34.264299,10.467384
4,33,C4-2,128119.0,693.761047,423.789062,19.189606,334152.0,639.0,758.0,369.0,...,0.067755,0.003850,0.087045,-0.429016,0.214191,0.994769,0.948297,0.076803,173.735453,50.955351
